In [1]:
## De functie die tf-records leest en ze uiteindelijk weer wegschrijft (maar dan met minder labels) heet readTfRecord
## Deze functie roept andere functies aan: getLabels en getLabelIndices

import tensorflow as tf
import numpy as np
import pandas as pd
import re
from tensorflow import gfile
#import readers
#import utils

/home/hugo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [30]:
train_data_pattern = '/home/hugo/jupyter-workspace/data/audioset/audioset_v1_embeddings/bal_train/*.tfrecord'
train_data_folder_new = '/home/hugo/jupyter-workspace/data/audioset/audioset_v1_embeddings/train_data_new/'

In [11]:
mid_to_label = pd.read_csv('audioset/class_labels_indices.csv')

,index,mid,display_name
0,0,/m/09x0r,Speech
1,1,/m/05zppz,"Male speech, man speaking"
2,2,/m/02zsn,"Female speech, woman speaking"
3,3,/m/0ytgt,"Child speech, kid speaking"
4,4,/m/01h8n0,Conversation
5,5,/m/02qldy,"Narration, monologue"
6,6,/m/0261r1,Babbling
7,7,/m/0brhx,Speech synthesizer
8,8,/m/07p6fty,Shout
9,9,/m/07q4ntr,Bellow


In [48]:
# Segments csv created Sun Mar  5 10:54:31 2017
# num_ytids=22160, num_segs=22160, num_unique_labels=527, num_positive_labels=52882
# YTID, start_seconds, end_seconds, positive_labels

vid_to_mid_balanced = pd.read_csv('audioset/balanced_train_segments.csv', skiprows=3, 
                                  quotechar='"', skipinitialspace = True, header=None,
                                 names = ["YTID", "start_seconds", "end_seconds", "positive_labels"])

vid_to_mid_balanced

,YTID,start_seconds,end_seconds,positive_labels
0,--PJHxphWEs,30.0,40.0,"/m/09x0r,/t/dd00088"
1,--ZhevVpy1s,50.0,60.0,/m/012xff
2,--aE2O5G5WE,0.0,10.0,"/m/03fwl,/m/04rlf,/m/09x0r"
3,--aO5cdqSAg,30.0,40.0,"/t/dd00003,/t/dd00005"
4,--aaILOrkII,200.0,210.0,"/m/032s66,/m/073cg4"
5,--cB2ZVjpnA,30.0,40.0,/m/01y3hg
6,--ekDLDTUXA,30.0,40.0,"/m/015lz1,/m/07pws3f"
7,-0DLPzsiXXE,30.0,40.0,"/m/04rlf,/m/07qwdck"
8,-0DdlOuIFUI,50.0,60.0,"/m/0130jx,/m/02jz0l,/m/0838f"
9,-0FHUc78Gqo,30.0,40.0,"/m/02w4v,/m/04rlf"


In [49]:
vid_to_mid_unbalanced = pd.read_csv('audioset/unbalanced_train_segments.csv', skiprows=3, 
                                   quotechar='"', skipinitialspace = True, header=None,
                                   names = ["YTID", "start_seconds", "end_seconds", "positive_labels"])

In [50]:
## Twee opties: 
# * balanced en unbalanced samen (zeer traag)
# * enkel balanced (bijna geen matches)

#vid_to_mid = pd.concat([vid_to_mid_balanced, vid_to_mid_unbalanced])
vid_to_mid = vid_to_mid_balanced
vid_to_mid

,YTID,start_seconds,end_seconds,positive_labels
0,--PJHxphWEs,30.0,40.0,"/m/09x0r,/t/dd00088"
1,--ZhevVpy1s,50.0,60.0,/m/012xff
2,--aE2O5G5WE,0.0,10.0,"/m/03fwl,/m/04rlf,/m/09x0r"
3,--aO5cdqSAg,30.0,40.0,"/t/dd00003,/t/dd00005"
4,--aaILOrkII,200.0,210.0,"/m/032s66,/m/073cg4"
5,--cB2ZVjpnA,30.0,40.0,/m/01y3hg
6,--ekDLDTUXA,30.0,40.0,"/m/015lz1,/m/07pws3f"
7,-0DLPzsiXXE,30.0,40.0,"/m/04rlf,/m/07qwdck"
8,-0DdlOuIFUI,50.0,60.0,"/m/0130jx,/m/02jz0l,/m/0838f"
9,-0FHUc78Gqo,30.0,40.0,"/m/02w4v,/m/04rlf"


In [7]:
#### Zoek bij een set m-id's de bijbehorende labels (mbv class_labels_indices.csv)

## Find labels for set of m-id's
## Input: cell of vid-to-mid tabel
## Output: list of unique labels belonging to mids and which are selected for our model

def getLabels(mid_str):
    
    ## Create a list of mids (instead of one string)
    mid_list = mid_str.split(',')

    ## For this list of m-id's, find all labels (that we want to include in our model)
    labels = []
    
    ## For each m-id, find labels, if any labels: add to label list
    for mid in mid_list:
        if (mid_to_label.loc[mid_to_label['mid'] == mid, 'in_model'] == 1).any():
            labels.append(mid_to_label.loc[mid_to_label['mid'] == mid, 'superclass'].values[0])
            
    ## Return unique set of labels
    return set(labels)

In [8]:
## Function that takes a set of labels and returns numbers, belonging to label
## Let op: dit zijn nog niet de labels die we uiteindelijk kiezen!! Het is puur even om te testen

def getLabelIndices(labels):
    label_indices = []
    
    if 'Speech' in labels:
        label_indices.append(1)
    if 'Music' in labels:
        label_indices.append(2)
    if 'Animal' in labels:
        label_indices.append(3)
    if 'Gunshots' in labels:
        label_indices.append(4)
    if 'Tools' in labels:
        label_indices.append(5)
    
    return label_indices

In [28]:
#### Lees tf-records en iterate over de examples
#### Voor elk example wordt de video-id gepakt en de bijbehorende lijst m-id's gezocht (in balanced_segment.csv)

## One tf-record has several examples
## This function checks for each example if it has a label we are interested in
## If so, we compute the one-hot vector
## Input: filename of tf-record
## Result: write new tf-record

def readTfRecord(tfrecords_filename):

    ## Keys in tf-record: labels, mean_audio, mean_rgb, video_id
    
    ## Create writer that writes new tf-record in train_data_folder_new
    path_tfrecord_new = str(train_data_folder_new + tfrecords_filename.split('\\')[1])
    writer = tf.python_io.TFRecordWriter(path_tfrecord_new)
        
    ## Iterator that goes through all examples of tf-record
    record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)

    for string_record in record_iterator:
        
        ## Take one example
        example = tf.train.Example()
        example.ParseFromString(string_record)
        
        ## Take the video id (substring -> [3:-2], because it contains some noise)
        video_id = str(example.features.feature['video_id'].bytes_list.value)[3:-2]
        
        ## Find mid-cell in table vid_to_mid for the video id
        ## YTID = vid, positive_label = mid
        mids = vid_to_mid.loc[vid_to_mid['YTID'].str.contains(video_id), 'positive_labels']
        
        ## If row exists, find labels for mids
        if mids.shape[0] > 0:
            labels = getLabels(str(mids))
            
            ## If mid has any labels (for our model), write new tf-record
            if len(labels) > 0:
                
                ## Find label index for each label (1=speech, 2=music etc)
                label_indices = getLabelIndices(labels)
                
                ## Bind the rgb-values and the label-indices and put them in the new example
                feature = {'label': tf.train.Feature(int64_list=tf.train.Int64List(value=label_indices)),
                           'mean_rgb': example.features.feature['mean_rgb']}
                example_new = tf.train.Example(features=tf.train.Features(feature=feature))
                
                ## Write the example
                writer.write(example.SerializeToString())
                print('Example gemaakt!')
                
    writer.close()

In [31]:
######################## This code will finally invoke all the other functions

## Read all train files
files = gfile.Glob(train_data_pattern)

for file in files:
    
    ## Read a file and rewrite it
    readTfRecord(file)

IndexError: list index out of range

In [82]:
files = gfile.Glob(train_data_pattern)


## Iterator that goes through all examples of tf-record
record_iterator = tf.python_io.tf_record_iterator(path=files[2])

for string_record in record_iterator:

    ## Take one example
    example = tf.train.Example()
    example.ParseFromString(string_record)
    video_id = str(example.features.feature['video_id'].bytes_list.value)[3:-2]
    #print(video_id)
    mids = vid_to_mid.loc[vid_to_mid['YTID'].str.contains(video_id), 'positive_labels']
    #print(vid_to_mid['YTID'].str.contains(video_id))
    #labels = getLabels(str(mids))
    print(str(mids))
    



9976    /m/01b_21,/m/04rlf,/m/09x0r,/m/0ytgt
Name: positive_labels, dtype: object
9975    /m/01jg02,/m/07p_0gm,/m/07rcgpl
Name: positive_labels, dtype: object
9974    /m/09x0r,/m/0k65p,/m/0l15bq,/m/0ytgt
Name: positive_labels, dtype: object
9972    /m/015lz1,/m/02w4v,/m/04rlf,/t/dd00032
Name: positive_labels, dtype: object
9978    /m/03qjg,/m/085jw
Name: positive_labels, dtype: object
9977    /m/01xqw,/m/02fsn,/m/0l14_3
Name: positive_labels, dtype: object
9973    /m/02bm9n,/m/05zc1,/m/0l7xg,/t/dd00077
Name: positive_labels, dtype: object
9971    /m/04rlf,/m/07rn7sz,/m/09x0r
Name: positive_labels, dtype: object


In [83]:
#vid_to_mid['YTID']

#video_id

#vid_to_mid['YTID']vid_to_mid['YTID'].str.contains(video_id)

example
#vid_to_mid.loc[1,'positive_labels']



features {
  feature {
    key: "end_time_seconds"
    value {
      float_list {
        value: 80.0
      }
    }
  }
  feature {
    key: "labels"
    value {
      int64_list {
        value: 0
        value: 137
        value: 443
      }
    }
  }
  feature {
    key: "start_time_seconds"
    value {
      float_list {
        value: 70.0
      }
    }
  }
  feature {
    key: "video_id"
    value {
      bytes_list {
        value: "P4BY72zK-aM"
      }
    }
  }
}